<a href="https://colab.research.google.com/github/jskaza/nfl-big-data-bowl-2023/blob/master/sack_probability_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using ReSaP and ReHaP to Predict Pass Rusher Impact as Plays Develop
*ReSaP: **Re**current **Sa**ck **P**robabilities*

*ReHaP: **Re**current **Ha**voc **P**robabilities*

**Jon Skaza & Matt Guthrie**

In [75]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, precision_recall_curve, PrecisionRecallDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
try:
  from google.colab import drive
  in_colab = True
except:
  in_colab = False
seed = 314 # for reproducibility, used in various places

## Dataset Preprocessing

In [83]:
if in_colab:
  drive.mount("/content/drive")
  path = "/content/drive/MyDrive/nfl-big-data-bowl-2023"
else:
  path = os.environ.get("BIG_DATA_BOWL")

df = pd.read_csv(f"{path}/data/dataset.csv", index_col=0).head(30000)
df.head()

,game_id,play_id,nfl_id,speed,pff_sack,havoc,x,y,dist_from_qb,qb_in_tackle_box,...,receiver_sep_1,receiver_sep_2,receiver_sep_3,receiver_sep_4,receiver_sep_5,quarter,down,yards_to_go,absolute_yardline_number,score_delta
1,2021090900,97,41263,0.96,0,1,1.74,-5.03,7.802083,1.0,...,3.030017,2.706917,6.139422,1.377679,4.278247,1,3,2,43.0,0
2,2021090900,97,41263,1.08,0,1,1.63,-5.01,7.766557,1.0,...,2.961689,2.659568,6.040149,1.369708,4.222345,1,3,2,43.0,0
3,2021090900,97,41263,1.30,0,1,1.47,-4.99,7.695193,1.0,...,2.859266,2.607221,5.928642,1.388416,3.898166,1,3,2,43.0,0
4,2021090900,97,41263,1.48,0,1,1.31,-4.94,7.603138,1.0,...,2.719577,2.452305,5.756813,1.424430,3.516049,1,3,2,43.0,0
5,2021090900,97,41263,2.16,0,1,1.04,-4.83,7.404627,1.0,...,2.612279,2.297325,5.472961,1.480034,3.040066,1,3,2,43.0,0


In [77]:
# examine missingness, models will need balanced sequences
df.info()
# get coords of 5 linemen
# nblockers

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   game_id                   5000 non-null   int64  
 1   play_id                   5000 non-null   int64  
 2   nfl_id                    5000 non-null   int64  
 3   speed                     5000 non-null   float64
 4   pff_sack                  5000 non-null   int64  
 5   havoc                     5000 non-null   int64  
 6   x                         5000 non-null   float64
 7   y                         5000 non-null   float64
 8   dist_from_qb              5000 non-null   float64
 9   qb_in_tackle_box          5000 non-null   float64
 10  x_blocker_1               5000 non-null   float64
 11  x_blocker_2               5000 non-null   float64
 12  x_blocker_3               5000 non-null   float64
 13  x_blocker_4               5000 non-null   float64
 14  x_blocke

In [104]:
def make_features(df: pd.DataFrame, group_by: list, feats: list, outcomes: list, naive = False):
  # check this
  X, y  = [], []
  grouped_df = df.groupby(group_by)
  for _, group_df in grouped_df:
    if naive:
      X.append(group_df[feats].to_numpy())
      y.append(group_df[outcomes])
    else:
      X.append(group_df[feats].fillna(-99.).to_numpy())
      y.append(group_df[outcomes].to_numpy()[0])
  
  # let's try two things for sensitivity:
  # chunk the test set, so we have various 1:5, 1:2, 1:11...
  # chunk the  training as well (to see if we actually have to train them on chunks)
  # oversampling

  if naive:
    X = np.concatenate(X)
    indices = pd.isnull(X).any(axis=0)
    X = np.delete(X, indices, axis=1)
    y = np.concatenate(y).ravel()
  else:
    X = tf.keras.utils.pad_sequences(X, dtype="float", padding="pre", value = -99)
    y = tf.keras.utils.pad_sequences(y, dtype="float", padding="pre", value= -99)

  return X, y

## ReSaP: **Re**current **Sa**ck **P**robabilities

In [105]:
model_metrics = {}
def add_metrics(model, outcome_name: str, model_name: str, X_test: np.ndarray, y_test: np.ndarray):
    global model_metrics
    model_metrics[outcome_name] = {}
    y_pred = model.predict(X_test)
    if type(model) == LogisticRegression:
        y_score = model.decision_function(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_score, pos_label = 1)
        roc = RocCurveDisplay(fpr = fpr, tpr = tpr)
        prec, recall, _ = precision_recall_curve(y_test, y_score, pos_label= 1)
        pr = PrecisionRecallDisplay(precision=prec, recall=recall)
        auc = roc_auc_score(y_test, y_score)
        model_metrics[outcome_name][model_name] = {"auc": auc, "roc_curve": roc, "pr_curve": pr}
    else:
        evaluation = model.evaluate(X_test, y_test)
        auc = evaluation[1]
        acc = evaluation[2]
        model_metrics[outcome_name][model_name] = {"auc": auc, "acc": acc}

### "No Sack" Pediction

### "Naive" Logistic Model

In [106]:
outcomes = ["pff_sack", "havoc"]
datasets = {}
for o in outcomes:
    group_by = ["game_id", "play_id", "nfl_id"]
    outcome = [o]
    feats = [x for x in list(df.columns) if x not in group_by + outcomes]

    X, y = make_features(df, group_by, feats, outcome, naive = True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

    weight_0 = (1 / sum(y_train == [0])) * (len(y_train) / 2.0)
    weight_1 = (1 / sum(y_train == [1])) * (len(y_train) / 2.0)
    class_weight = {0: weight_0, 1: weight_1}

    datasets[o] = {"X_train": X_train, "X_test": X_test,
    "y_train": y_train, "y_test": y_test, "class_weight": class_weight}

In [107]:
for k, v in datasets.items():
    log_reg = LogisticRegression(class_weight = v["class_weight"], max_iter = 1000)
    log_reg.fit(v["X_train"], v["y_train"])
    add_metrics(log_reg, k, "logistic", v["X_test"], v["y_test"])

In [108]:
model_metrics

{'pff_sack': {'logistic': {'auc': 0.9106793294280415,
   'roc_curve': <sklearn.metrics._plot.roc_curve.RocCurveDisplay at 0x7f860eb3e4d0>,
   'pr_curve': <sklearn.metrics._plot.precision_recall_curve.PrecisionRecallDisplay at 0x7f8760ae5cf0>}},
 'havoc': {'logistic': {'auc': 0.7517986627742727,
   'roc_curve': <sklearn.metrics._plot.roc_curve.RocCurveDisplay at 0x7f8495295570>,
   'pr_curve': <sklearn.metrics._plot.precision_recall_curve.PrecisionRecallDisplay at 0x7f848cbfe5c0>}}}

### LSTM

In [ ]:
outcomes = ["pff_sack", "havoc"]
datasets = {}
for o in outcomes:
    group_by = ["game_id", "play_id", "nfl_id"]
    outcome = [o]
    feats = [x for x in list(df.columns) if x not in group_by + outcomes]

    X, y = make_features(df, group_by, feats, outcome)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

    weight_0 = (1 / sum(y_train == [0])) * (len(y_train) / 2.0)
    weight_1 = (1 / sum(y_train == [1])) * (len(y_train) / 2.0)
    class_weight = {0: weight_0, 1: weight_1}

    num_epochs = 1
    val = 0.2 

    datasets[o] = {"X_train": X_train, "X_test": X_test,
    "y_train": y_train, "y_test": y_test, "class_weight": class_weight,
    "num_epochs": num_epochs, "val": val}


In [ ]:
for k, v in datasets.items():

    tf.random.set_seed(seed)
    np.random.seed(seed)
    
    lstm_mod = tf.keras.Sequential()
    lstm_mod.add(tf.keras.layers.Masking(mask_value= -99., input_shape= v["X_train"].shape[1:]))
    lstm_mod.add(tf.keras.layers.LSTM(128, input_shape = v["X_train"].shape[1:]))
    lstm_mod.add(tf.keras.layers.Dense(v["y_train"].shape[1], activation="sigmoid"))

    lstm_mod.compile(loss = "binary_crossentropy", optimizer="adam",
     metrics = [tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()])

    callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5, restore_best_weights=True)]

    lstm_mod.fit(v["X_train"], v["y_train"], epochs = v["num_epochs"], 
    validation_split = v["val"], class_weight = v["class_weight"], callbacks = callbacks)
    
    add_metrics(lstm_mod, k, "lstm", v["X_test"], v["y_test"])

Epoch 1/50
688/688 [==============================] - 216s 296ms/step - loss: 0.6973 - auc_2: 0.5931 - binary_accuracy: 0.8039 - val_loss: 0.6931 - val_auc_2: 0.8461 - val_binary_accuracy: 0.5128
Epoch 2/50
327/688 [=============>................] - ETA: 1:38 - loss: 0.5740 - auc_2: 0.8478 - binary_accuracy: 0.7922

KeyboardInterrupt: 

In [ ]:
# roc curve
# accuracy: predict no sack
# logistic 
# lstm
# break test into chunks
model_metrics

### Transformer

In [86]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = tf.keras.layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = tf.keras.layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [87]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    lstm_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    x = tf.keras.layers.Masking(mask_value=-99.,input_shape= input_shape)(x)
    x = tf.keras.layers.LSTM(lstm_units, input_shape = input_shape, return_sequences=True)(x)

    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    
    for dim in mlp_units:
        x = tf.keras.layers.Dense(dim, activation="relu")(x)
        x = tf.keras.layers.Dropout(mlp_dropout)(x)
    
    outputs = tf.keras.layers.Dense(y.shape[1], activation="sigmoid")(x)
    return tf.keras.Model(inputs, outputs)

In [84]:
outcomes = ["pff_sack", "havoc"]
datasets = {}
for o in outcomes:
    group_by = ["game_id", "play_id", "nfl_id"]
    outcome = [o]
    feats = [x for x in list(df.columns) if x not in group_by + outcomes]

    X, y = make_features(df, group_by, feats, outcome)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

    weight_0 = (1 / sum(y_train == [0])) * (len(y_train) / 2.0)
    weight_1 = (1 / sum(y_train == [1])) * (len(y_train) / 2.0)
    class_weight = {0: weight_0, 1: weight_1}

    num_epochs = 1
    val = 0.2 

    datasets[o] = {"X_train": X_train, "X_test": X_test,
    "y_train": y_train, "y_test": y_test, "class_weight": class_weight,
    "num_epochs": num_epochs, "val": val}

In [89]:
for k, v in datasets.items():
    input_shape = v["X_train"].shape[1:]

    model = build_model(
        input_shape,
        head_size=128,
        num_heads=4,
        ff_dim=4,
        num_transformer_blocks=1,
        mlp_units=[128],
        mlp_dropout=0.2,
        dropout=0.25,
        lstm_units=32
    )

    model.compile(
        loss="binary_crossentropy",
        optimizer="adam",
        #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        metrics = [tf.keras.metrics.AUC()]
    )
    #model.summary()

    callbacks = [tf.keras.callbacks.EarlyStopping(min_delta=0.01, patience=3, restore_best_weights=True)]

    model.fit(
        v["X_train"],
        v["y_train"],
        validation_split=v["val"],
        epochs=v["num_epochs"],
        #batch_size=64,
        callbacks=callbacks,
        class_weight = v["class_weight"]
    )

    model.evaluate(v["X_test"], v["y_test"], verbose=1)

7/7 [==============================] - 6s 329ms/step - loss: 0.7361 - auc_6: 0.5833
